In [0]:
%pip install -r ../requirements.txt
%pip install -U -qq databricks-agents
%restart_python

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
%pip freeze > requirements_driver.txt 

In [0]:
import mlflow
from mlflow.models import ModelConfig
from src.utils import get_SP_credentials

mlflow.set_registry_uri('databricks-uc')

cfg = ModelConfig(development_config="config.yml")

client_id, client_secret = get_SP_credentials(
    scope='aichemy',
    client_id_key='client_id', #if retrieving secrets (but doesn't work with mlflow logging)
    client_secret_key='client_secret', #if retrieving secrets (but doesn't work with mlflow logging)
)

In [0]:
catalog_name = cfg.get("catalog")
schema_name = cfg.get("schema")
model_name = "multiagent"

registered_name = f"{catalog_name}.{schema_name}.{model_name}"
artifact_path = "agent"

In [0]:
from uuid import uuid4

query = "What is the mw of danuglipron?"
input_message = {
    "messages": [
        {
            "role": "user",
            "content": query
        }
    ]
}
input = {
    "input": input_message["messages"],
    "custom_inputs": {"thread_id": str(uuid4())}
}

## Log the `agent` as an MLflow model

In [0]:
# Log the model to MLflow
import os
from mlflow.models.resources import (
    DatabricksVectorSearchIndex, 
    DatabricksServingEndpoint,
    DatabricksFunction,
    DatabricksUCConnection,
    DatabricksGenieSpace,
    DatabricksTable,
    DatabricksSQLWarehouse,
    DatabricksLakebase
)

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        python_model=os.path.join(os.getcwd(), '5_multiagent'),
        name=artifact_path,
        registered_model_name=registered_name,
        model_config="config.yml",
        pip_requirements="../requirements.txt",
        code_paths = ["../src"],
        input_example=input,
        # specify resources for deployed server to have explicit access
        resources=[
            DatabricksServingEndpoint(endpoint_name=cfg.get("llm_endpoint")),
            DatabricksServingEndpoint(endpoint_name="databricks-gte-large-en"),
            DatabricksVectorSearchIndex(index_name=cfg.get('retriever')['vs_index']),
            DatabricksFunction(function_name=cfg.get("uc_functions")[0]),
            DatabricksUCConnection(connection_name=cfg.get("uc_connections")["pubchem"]),
            DatabricksUCConnection(connection_name=cfg.get("uc_connections")["opentargets"]),
            DatabricksGenieSpace(genie_space_id=cfg.get("genie_space_id")),
            DatabricksTable(table_name=cfg.get("genie_table")),
            DatabricksSQLWarehouse(warehouse_id="036e73c70fc97e1f"),
            DatabricksLakebase(database_instance_name=cfg.get("lakebase")["instance_name"]),
        ]
    )

## Test inferencing

In [0]:
#model_uri = f"runs:/{logged_agent_info.run_id}/{artifact_path}"
model_uri = 'runs:/54f60cfcde374f3dab3bbe5fbde9b867/agent' #v30
model_uri

In [0]:
loaded_model = mlflow.pyfunc.load_model(model_uri)
response = loaded_model.predict(input)
response

## Deploy model

In [0]:
from src.utils import get_latest_model_version

latest_version = get_latest_model_version('aichemy2_catalog.aichemy.customer_service')
latest_version

In [0]:
from databricks import agents

# Deploy the model to the review app and a model serving endpoint
agents.deploy(model_name='aichemy2_catalog.aichemy.customer_service', 
              model_version=latest_version,
              endpoint_name="aichemy",
              scale_to_zero=True,
              tags = {"endpointSource": "docs"},
              environment_vars={
                "MAX_MODEL_LOADING_TIMEOUT": "600",
                "DATABRICKS_CLIENT_ID": client_id,
                "DATABRICKS_CLIENT_SECRET": client_secret})